In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch

from peptdeep.pretrained_models import ModelManager

model_mgr = ModelManager(mask_modloss=True)

In [3]:
from peptdeep.model.rt import IRT_PEPTIDE_DF
IRT_PEPTIDE_DF['charge'] = 2
IRT_PEPTIDE_DF['nce'] = 30.0
IRT_PEPTIDE_DF['instrument'] = 'QE'
trace_df = IRT_PEPTIDE_DF.query('nAA==12')

### torch.jit.trace to compile 

`torch.jit.script()` does not allow global variables, *args and **kwargs, so it does not work for huggingface transformers.

But `torch.jit.trace()` works well.

### Trace RT model

In [4]:
features = model_mgr.rt_model._get_features_from_batch_df(
    trace_df
)
model_mgr.rt_model.model.eval()
traced_rt = torch.jit.trace(model_mgr.rt_model.model, features)

In [5]:
traced_rt(*model_mgr.rt_model._get_features_from_batch_df(
    IRT_PEPTIDE_DF.query('nAA==10')
))

tensor([0.4009], grad_fn=<SqueezeBackward1>)

### Trace CCS model

In [6]:
features = model_mgr.ccs_model._get_features_from_batch_df(
    trace_df
)
model_mgr.ccs_model.model.eval()
traced_ccs = torch.jit.trace(model_mgr.ccs_model.model, features)

In [7]:
traced_ccs(*model_mgr.ccs_model._get_features_from_batch_df(
    IRT_PEPTIDE_DF.query('nAA==10')
))

tensor([364.8280], grad_fn=<SqueezeBackward1>)

### Trace MS2 model

In [8]:
features = model_mgr.ms2_model._get_features_from_batch_df(
    trace_df
)
model_mgr.ms2_model.model.eval()
traced_ms2=torch.jit.trace(model_mgr.ms2_model.model, features)

/Users/zengwenfeng/Workspace/alphapeptdeep/peptdeep/model/building_block.py:422: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return x + self.pe[:,:x.size(1),:]


In [9]:
traced_ms2(*model_mgr.ms2_model._get_features_from_batch_df(
    IRT_PEPTIDE_DF.query('nAA==10')
)).size()

torch.Size([1, 9, 8])

### Save traced model

In [10]:
# torch.jit.save(traced_ms2, "traced_ms2.pt")
# torch.jit.save(traced_rt, "traced_rt.pt")
# torch.jit.save(traced_ccs, "traced_ccs.pt")

In [11]:
# traced_model = torch.jit.load(...)